# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [176]:
#load python packages
import os
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [177]:
os.getcwd()
path = '/Users/omniverse/Documents/Springboard/Capstone/PreProcessing/step4'
df = pd.read_csv('/Users/omniverse/Documents/Springboard/Capstone/PreProcessing/step4/step3_output.csv')
del df['Unnamed: 0']
df.head()

,Name,Region,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,Alaska,Alaska,2090,294,1796,0,0.0,0,0,...,30.0,30.0,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,1
1,Sunrise Park Resort,Arizona,Arizona,11100,1800,9200,0,0.0,0,1,...,800.0,80.0,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,2
2,Yosemite Ski & Snowboard Area,Northern California,California,7800,600,7200,0,0.0,0,0,...,88.0,0.0,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,0
3,Boreal Mountain Resort,Sierra Nevada,California,7700,500,7200,0,0.0,0,1,...,380.0,200.0,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,0
4,Dodge Ridge,Sierra Nevada,California,8200,1600,6600,0,0.0,0,0,...,862.0,0.0,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,0


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [178]:
statedum =df.select_dtypes(include=['object']) 
df = pd.concat([df.drop(statedum, axis=1), pd.get_dummies(statedum)], axis=1)
df.head()

,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,state_Rhode Island,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,2090,294,1796,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,11100,1800,9200,0,0.0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0
2,7800,600,7200,0,0.0,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
3,7700,500,7200,0,0.0,0,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
4,8200,1600,6600,0,0.0,0,0,1,2,5,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [179]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
# I didn't include the 'Name' variable as it cannot find it.
X = df.drop(['AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.loc[:,'AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [180]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [181]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [182]:
# Make a variable called y_pred and assign it the result of calling predict() 
#on our model variable with parameter X_test

y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [183]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [184]:
explained_variance_score(y_test, y_pred)

0.6272327196280798

In [185]:
mean_absolute_error(y_test, y_pred)

6.793821354394897

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [186]:
lm.intercept_

56.50856592955289

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [187]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
# pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
coeffdf = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

#coeffdf.loc[:, 'Coefficient'].sort_values(ascending=False).head(10)

coeffdf.head(10)

,Coefficient
summit_elev,0.319703
vertical_drop,1.452503
base_elev,0.022863
trams,0.263160
fastEight,0.164692
fastSixes,0.114376
fastQuads,1.020926
quad,0.690219
triple,0.908965
double,0.054913


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [207]:
#df = pd.read_csv('step3_output.csv', index_col=0)
df = pd.read_csv('/Users/omniverse/Documents/Springboard/Capstone/PreProcessing/step4/step3_output.csv', index_col=1)
del df['Unnamed: 0']
#df.drop('Hilltop Ski Area', inplace=True)
df.drop(['AdultWeekday','vertical_drop', 'Region', 'state'], axis=1, inplace=True)
df


,summit_elev,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,...,LongestRun_mi,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
Name,,,,,,,,,,,,,,,,,,,,,
Hilltop Ski Area,2090,1796,0,0.0,0,0,0,1,0,2,...,1.0,30.0,30.0,150.000000,36.0,69.0,34.00000,152.000000,30.0,1
Sunrise Park Resort,11100,9200,0,0.0,0,1,2,3,1,0,...,1.2,800.0,80.0,115.000000,49.0,250.0,78.00000,104.000000,80.0,2
Yosemite Ski & Snowboard Area,7800,7200,0,0.0,0,0,0,1,3,1,...,0.4,88.0,0.0,110.000000,84.0,300.0,47.00000,107.000000,0.0,0
Boreal Mountain Resort,7700,7200,0,0.0,0,1,1,3,1,2,...,1.0,380.0,200.0,150.000000,54.0,400.0,64.16681,150.000000,200.0,0
Dodge Ridge,8200,6600,0,0.0,0,0,1,2,5,4,...,2.0,862.0,0.0,115.103943,69.0,350.0,78.00000,140.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hogadon Basin,8000,7400,0,0.0,0,0,0,0,1,1,...,0.6,92.0,32.0,121.000000,61.0,80.0,48.00000,95.000000,0.0,0
Sleeping Giant Ski Resort,7428,6619,0,0.0,0,0,0,1,1,1,...,1.0,184.0,18.0,61.000000,81.0,310.0,42.00000,77.000000,0.0,0
Snow King Resort,7808,6237,0,0.0,0,0,1,1,1,0,...,1.0,400.0,250.0,121.000000,80.0,300.0,59.00000,123.000000,110.0,0


In [208]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
# I didn't include the 'Name' variable as it cannot find it.
X = df

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.loc[:,'AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [209]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [210]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

In [211]:
y_pred = model.predict(X_test)

In [212]:
explained_variance_score(y_test, y_pred)

1.0

In [213]:
mean_absolute_error(y_test, y_pred)

3.633457171500512e-14

In [214]:
lm.intercept_

56.927323639622024

**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [216]:
#df = pd.read_csv('step3_output.csv', index_col=0)
df = pd.read_csv('/Users/omniverse/Documents/Springboard/Capstone/PreProcessing/step4/step3_output.csv', index_col=1)
del df['Unnamed: 0']
#df.drop('Hilltop Ski Area', inplace=True)
df.drop(['AdultWeekday','vertical_drop', 'Region', 'state', 'summit_elev', 'base_elev'], axis=1, inplace=True)
df



,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,total_chairs,Runs,...,LongestRun_mi,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
Name,,,,,,,,,,,,,,,,,,,,,
Hilltop Ski Area,0,0.0,0,0,0,1,0,2,3,13.0,...,1.0,30.0,30.0,150.000000,36.0,69.0,34.00000,152.000000,30.0,1
Sunrise Park Resort,0,0.0,0,1,2,3,1,0,7,65.0,...,1.2,800.0,80.0,115.000000,49.0,250.0,78.00000,104.000000,80.0,2
Yosemite Ski & Snowboard Area,0,0.0,0,0,0,1,3,1,5,10.0,...,0.4,88.0,0.0,110.000000,84.0,300.0,47.00000,107.000000,0.0,0
Boreal Mountain Resort,0,0.0,0,1,1,3,1,2,8,33.0,...,1.0,380.0,200.0,150.000000,54.0,400.0,64.16681,150.000000,200.0,0
Dodge Ridge,0,0.0,0,0,1,2,5,4,12,67.0,...,2.0,862.0,0.0,115.103943,69.0,350.0,78.00000,140.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hogadon Basin,0,0.0,0,0,0,0,1,1,2,28.0,...,0.6,92.0,32.0,121.000000,61.0,80.0,48.00000,95.000000,0.0,0
Sleeping Giant Ski Resort,0,0.0,0,0,0,1,1,1,3,48.0,...,1.0,184.0,18.0,61.000000,81.0,310.0,42.00000,77.000000,0.0,0
Snow King Resort,0,0.0,0,0,1,1,1,0,3,32.0,...,1.0,400.0,250.0,121.000000,80.0,300.0,59.00000,123.000000,110.0,0


In [218]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
# I didn't include the 'Name' variable as it cannot find it.
X = df

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.loc[:,'AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [219]:
y_pred = model.predict(X_test)

In [220]:
explained_variance_score(y_test, y_pred)

1.0

In [221]:
mean_absolute_error(y_test, y_pred)

3.633457171500512e-14

In [222]:
lm.intercept_

56.927323639622024

## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection:

In [0]:
1